In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


# Load Dataset
df = pd.read_csv("../dataset/tiny_chat_dataset_large.csv")
df.head()


# Prepare Text Data

# Automatically detect likely column names
possible_user_cols = ['user', 'input', 'input_text', 'prompt', 'query']
possible_bot_cols  = ['bot', 'reply', 'reply_text', 'response', 'output']

user_col = next((c for c in df.columns if c.lower() in possible_user_cols), None)
bot_col  = next((c for c in df.columns if c.lower() in possible_bot_cols), None)

if user_col is None or bot_col is None:
    raise ValueError(f"Could not find valid user/bot columns in CSV. Found columns: {df.columns.tolist()}")

print(f"Using columns → User: '{user_col}' | Bot: '{bot_col}'")

# Convert to lists
user_texts = df[user_col].astype(str).tolist()
bot_texts  = ["<start> " + str(t) + " <end>" for t in df[bot_col].astype(str).tolist()]

✅ Using columns → User: 'input_text' | Bot: 'reply_text'
Vocabulary size: 154
Input shape: (18000, 15)
Target input shape: (18000, 14)
Target output shape: (18000, 14, 1)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 64)     9856        ['input_7[0][0]']                
                                                                                                  
 input_8 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 gru_8 (GRU)        

In [ ]:
# Tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(user_texts + bot_texts)
VOCAB_SIZE = len(tokenizer.word_index) + 1
print("Vocabulary size:", VOCAB_SIZE)

# Convert to token sequences
input_seq = tokenizer.texts_to_sequences(user_texts)
target_seq = tokenizer.texts_to_sequences(bot_texts)

# Pad sequences
MAX_LEN = 15
input_seq = pad_sequences(input_seq, maxlen=MAX_LEN, padding='post')
target_seq = pad_sequences(target_seq, maxlen=MAX_LEN, padding='post')

# Shift targets for teacher forcing
target_input = target_seq[:, :-1]
target_output = target_seq[:, 1:]
target_output = np.expand_dims(target_output, -1)

print("Input shape:", input_seq.shape)
print("Target input shape:", target_input.shape)
print("Target output shape:", target_output.shape)


# Define Seq2Seq Model with Attention
EMBED_DIM = 64
HIDDEN_DIM = 128

# Encoder
encoder_inputs = keras.Input(shape=(None,))
enc_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h = layers.GRU(HIDDEN_DIM, return_sequences=True, return_state=True)(enc_emb)

# Decoder
decoder_inputs = keras.Input(shape=(None,))
dec_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(decoder_inputs)
decoder_gru = layers.GRU(HIDDEN_DIM, return_sequences=True, return_state=True)

# Attention

# Project encoder output to decoder embedding dimension
encoder_proj = layers.Dense(64)(encoder_outputs)

# Apply attention (queries = decoder embeddings, values = projected encoder outputs)
attn = layers.Attention()
context = attn([dec_emb, encoder_proj])

# Combine context + decoder embeddings
decoder_combined = layers.Concatenate(axis=-1)([dec_emb, context])

decoder_outputs, _ = decoder_gru(decoder_combined, initial_state=state_h)
decoder_dense = layers.Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


# Train the Model
history = model.fit(
    [input_seq, target_input],
    target_output,
    batch_size=32,
    epochs=200,
    validation_split=0.1,
    verbose=1
)

# Save Model + Tokenizer
model.save('tiny_bot_attn.h5')
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Model training completed and saved successfully!")

# Define Inference Function

reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}

In [ ]:
def generate_reply(input_text, max_len=15):
    seq = tokenizer.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=max_len, padding='post')
    
    enc_out, enc_state = model.layers[3].output, model.layers[4].output
    
    pred_seq = [tokenizer.word_index['<start>']]
    
    for _ in range(max_len):
        dec_in = pad_sequences([pred_seq], maxlen=max_len, padding='post')
        preds = model.predict([seq, dec_in], verbose=0)
        next_token = np.argmax(preds[0, len(pred_seq)-1])
        pred_seq.append(next_token)
        if reverse_word_index.get(next_token) == '<end>':
            break

    response = ' '.join([reverse_word_index.get(i, '') for i in pred_seq if i > 0])
    response = response.replace('<start>', '').replace('<end>', '').strip()
    return response

In [28]:
def chat(query):
    ans = generate_reply(query)
    print(f"User: {query} --> Bot: {ans}")

In [29]:
chat("Hi")
chat("Hello there")
chat("Good morning TinyBot")
chat("How are you?")
chat("What's your name?")
chat("What can you do?")
chat("Turn on the light")
chat("Switch on the fan")
chat("Activate the dim light")
chat("Turn off the curtain")
chat("Open the curtain")
chat("Close the door")
chat("Is the fridge on?")
chat("Check the door sensor")
chat("Show me the camera")
chat("Is the light off?")
chat("It's too hot")
chat("It's dark")
chat("The plants are dry")
chat("The aquarium looks dirty")
chat("Start the hydroponic farm motor")
chat("Stop the hydroponic farm motor")
chat("Switch off the refrigerator")
chat("Turn off everything")
chat("Enable the fan for 10 minutes")
chat("Schedule the light at 7 PM")
chat("What’s the room temperature?")
chat("What’s the humidity?")
chat("Thanks")
chat("Thank you TinyBot")
chat("Bye")
chat("Good night")
chat("See you later")
chat("You are smart")
chat("You are dumb")
chat("Who made you?")
chat("Tell me a joke")
chat("How’s the day?")
chat("I’m bored")
chat("Can you dance?")
chat("Sing a song")
chat("Restart the system")
chat("What devices are connected?")
chat("List all devices")
chat("Can you check my home?")
chat("Report status")
chat("Everything okay?")
chat("Are you online?")
chat("Good evening")
chat("Turn on the fan and light")
chat("Turn off fan and fridge")


User: Hi --> Bot: nice to see you!
User: Hello there --> Bot: hi!
User: Good morning TinyBot --> Bot: bye!
User: How are you? --> Bot: doing great! just monitoring your devices.
User: What's your name? --> Bot: hi, i’m tinybot!
User: What can you do? --> Bot: checking light status...
User: Turn on the light --> Bot: okay turning on the light.
User: Switch on the fan --> Bot: sure! activating the fan.
User: Activate the dim light --> Bot: okay turning on the dim light.
User: Turn off the curtain --> Bot: done, curtain is off.
User: Open the curtain --> Bot: okay, deactivating the curtain.
User: Close the door --> Bot: done, refrigerator is off.
User: Is the fridge on? --> Bot: it’s always time to automate something!
User: Check the door sensor --> Bot: the door sensor is currently on.
User: Show me the camera --> Bot: the cctv camera is currently on.
User: Is the light off? --> Bot: the light is currently off.
User: It's too hot --> Bot: no problem!
User: It's dark --> Bot: no problem!
